Use this utlity to update the returns and std_dev fields within investment-options.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import brownbear as bb

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [3]:
# set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [5]:
# read in sp500.csv
sp500 = pd.read_csv('sp500.csv')
sp500.drop(columns=['SEC filings', 'Headquarters Location', 'Date first added','CIK', 'Founded'], inplace=True)
sp500.rename(columns={'Security':'Description',
                      'GICS Sector':'Asset Class'}, inplace=True)
sp500.set_index("Symbol", inplace=True)
sp500

,Description,Asset Class,GICS Sub-Industry
Symbol,,,
MMM,3M Company,Industrials,Industrial Conglomerates
ABT,Abbott Laboratories,Health Care,Health Care Equipment
ABBV,AbbVie Inc.,Health Care,Pharmaceuticals
ABMD,Abiomed,Health Care,Health Care Equipment
ACN,Accenture,Information Technology,IT Consulting & Other Services
...,...,...,...
YUM,Yum! Brands Inc,Consumer Discretionary,Restaurants
ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments
ZBH,Zimmer Biomet,Health Care,Health Care Equipment


In [6]:
# read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Energy': 'US Stocks:Energy',
 'Materials': 'US Stocks:Materials',
 'Industrials': 'US Stocks:Industrials',
 'Consumer Discretionary': 'US Stocks:Consumer Discretionary',
 'Consumer Staples': 'US Stocks:Consumer Staples',
 'Health Care': 'US Stocks:Healthcare',
 'Financials': 'US Stocks:Financials',
 'Information Technology': 'US Stocks:Technology',
 'Communication Services': 'US Stocks:Communication Services',
 'Utilities': 'US Stocks:Utilities',
 'Real Estate': 'US Stocks:Real Estate'}

In [7]:
# map sp500 GICS sectors to brownbear defined asset classes
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

sp500['Asset Class'] = sp500.apply(_asset_class, axis=1)

# yahoo finance uses '-' where '.' is used in symbol names
sp500.index = sp500.index.str.replace('.', '-')
sp500

,Description,Asset Class,GICS Sub-Industry
Symbol,,,
MMM,3M Company,US Stocks:Industrials,Industrial Conglomerates
ABT,Abbott Laboratories,US Stocks:Healthcare,Health Care Equipment
ABBV,AbbVie Inc.,US Stocks:Healthcare,Pharmaceuticals
ABMD,Abiomed,US Stocks:Healthcare,Health Care Equipment
ACN,Accenture,US Stocks:Technology,IT Consulting & Other Services
...,...,...,...
YUM,Yum! Brands Inc,US Stocks:Consumer Discretionary,Restaurants
ZBRA,Zebra Technologies,US Stocks:Technology,Electronic Equipment & Instruments
ZBH,Zimmer Biomet,US Stocks:Healthcare,Health Care Equipment


In [8]:
# make symbols list
symbols = list(sp500.index)
#symbols

In [9]:
# get the timeseries for the symbols and compile into a single csv
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

.........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................


In [10]:
# read symbols timeseries into a dataframe
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df = df[:]
df.tail()[['NVDA', 'WST', 'ADBE', 'NOW', 'AMD', 'PYPL', 'AAPL', 'MSFT', 'DHR', 'TMO', 'AMZN', 'DXCM', 'ODFL', 'REGN', 'ROL', 'FDX', 'TMUS', 'CDNS', 'CRM', 'MSCI']]

,NVDA,WST,ADBE,NOW,AMD,PYPL,AAPL,MSFT,DHR,TMO,AMZN,DXCM,ODFL,REGN,ROL,FDX,TMUS,CDNS,CRM,MSCI
Date,,,,,,,,,,,,,,,,,,,,
2021-05-28,649.78,347.51,504.58,473.88,80.08,260.02,124.61,249.68,256.14,469.50,3223.07,369.39,265.25,502.43,34.09,314.81,141.45,126.99,238.10,468.13
2021-06-01,650.58,340.16,495.91,469.17,80.81,259.27,124.28,247.40,245.21,447.01,3218.65,365.67,267.85,503.23,33.76,310.85,141.70,125.75,236.20,461.97
2021-06-02,671.13,333.77,495.77,467.01,81.97,262.17,125.06,247.30,242.01,449.21,3233.99,370.99,263.42,505.36,33.92,306.57,140.98,124.53,234.62,468.87
2021-06-03,678.79,332.77,493.14,459.20,80.28,257.79,123.54,245.71,241.01,445.33,3187.01,376.74,264.18,505.66,33.09,306.31,142.32,123.94,230.84,461.98
2021-06-04,703.13,333.43,504.50,460.65,81.58,263.04,125.89,250.79,242.27,448.76,3206.22,382.91,260.27,511.89,33.19,302.12,144.51,126.75,237.48,466.36


In [11]:
# sample symbol
symbol = 'MMM'

In [12]:
annual_returns = bb.annualize_returns(df, timeperiod='daily', years=1)
annual_returns[symbol]

31.041818758950246

In [13]:
# calculate annualized returns
annual_returns_1mo = bb.annualize_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualize_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualize_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualize_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualize_returns(df, timeperiod='daily', years=5)

In [14]:
# calculate volatility
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola[symbol]

0.16655976817076731

In [15]:
# calculate downside volatility
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola[symbol]

0.10171409149432402

In [16]:
# resample df on a monthly basis
df.index = pd.to_datetime(df.index)
monthly = df.resample('M').ffill()
bb.print_full(monthly[symbol])

Date
2015-01-31   134.61
2015-02-28   140.75
2015-03-31   137.66
2015-04-30   130.52
2015-05-31   133.60
2015-06-30   129.59
2015-07-31   127.10
2015-08-31   120.20
2015-09-30   119.89
2015-10-31   132.95
2015-11-30   133.29
2015-12-31   128.23
2016-01-31   128.54
2016-02-29   134.50
2016-03-31   142.87
2016-04-30   143.51
2016-05-31   145.28
2016-06-30   151.14
2016-07-31   153.94
2016-08-31   155.66
2016-09-30   153.05
2016-10-31   143.56
2016-11-30   150.11
2016-12-31   156.08
2017-01-31   152.80
2017-02-28   163.94
2017-03-31   168.32
2017-04-30   172.28
2017-05-31   180.95
2017-06-30   184.24
2017-07-31   178.03
2017-08-31   181.86
2017-09-30   186.83
2017-10-31   204.89
2017-11-30   217.50
2017-12-31   210.55
2018-01-31   224.09
2018-02-28   211.92
2018-03-31   197.53
2018-04-30   174.92
2018-05-31   178.69
2018-06-30   178.23
2018-07-31   192.36
2018-08-31   192.37
2018-09-30   192.18
2018-10-31   173.53
2018-11-30   190.93
2018-12-31   174.97
2019-01-31   183.93
2019-02-28   19

In [17]:
# calculate monthly returns
monthly_returns = monthly.pct_change()
monthly_returns[symbol]

Date
2015-01-31     nan
2015-02-28    0.05
2015-03-31   -0.02
2015-04-30   -0.05
2015-05-31    0.02
              ... 
2021-02-28    0.00
2021-03-31    0.10
2021-04-30    0.02
2021-05-31    0.04
2021-06-30    0.01
Freq: M, Name: MMM, Length: 78, dtype: float64

In [18]:
# calculate standard deviation
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev[symbol]

0.22058470745508066

In [19]:
# read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
lines

['# Description: S&P 500 investment options. 09/29/2020',
 '',
 '# Format',
 '"Investment Option","Description","Asset Class","1 mo","3 mo","1 Yr","3 Yr","5 Yr","Vola","DS Vola","Std Dev"',
 '# Note: "Description" field is optional']

In [20]:
# for each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev
out = lines.copy()

# This is still slow (2.53 s)
for i, (index, row) in enumerate(sp500.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append(
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
        .format(symbol, description, asset_class,
                ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [21]:
# write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')